# Data Wrangling

This notebook will contain all of the code used to clean the data within a Pandas dataframe and engineer new features.

In [21]:
import pandas as pd
import geopy.distance
from geopy.extra.rate_limiter import RateLimiter
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../data/listings.csv")

In [3]:
df.shape

(74840, 74)

## Data Cleaning

List of all columns within the dataset

In [4]:
list(df.columns)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availa

In [5]:
# dropping columns that are based on review data

review_cols_drop = ['number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
                    'first_review', 'last_review', 'review_scores_rating',
                    'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                    'review_scores_communication', 'review_scores_location',
                    'review_scores_value', 'reviews_per_month']

df.drop(review_cols_drop,axis=1,inplace=True)

In [6]:
# dropping columns related to the host that I won't be using as features

host_cols_drop = ['host_url', 'host_name', 'host_location',
                  'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
                  'host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count_entire_homes',
                  'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms',]

df.drop(host_cols_drop,axis=1,inplace=True)

In [7]:
# dropping columns that can't or wouldn't be useful as predictor variables

useless_cols_drop = ['scrape_id', 'last_scraped', 'picture_url', 'neighbourhood',
                     'neighbourhood_group_cleansed', 'bathrooms', 'minimum_nights',
                     'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
                     'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm',
                     'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
                     'availability_30', 'availability_60', 'availability_90', 'availability_365',
                     'calendar_last_scraped', 'license']

df.drop(useless_cols_drop,axis=1,inplace=True)

Let's have a look at the variables with missing values

In [9]:
null_values = pd.DataFrame(df.isnull().sum())
null_values = null_values[null_values[0] != 0]
null_values

,0
name,23
description,3126
neighborhood_overview,29439
host_since,1981
host_about,34283
host_response_time,41905
host_response_rate,41905
host_acceptance_rate,39343
host_is_superhost,1981
host_has_profile_pic,1981


The following text variables all have blank values in their columns. I'm going to fill them in with the word 'null' to avoid having to remove these properties from dataset.

In [10]:
# filling the null values in the following columns rather than removing any rows

df.name.fillna('null',inplace=True)
df.description.fillna('null',inplace=True)
df.neighborhood_overview.fillna('null',inplace=True)
df.host_about.fillna('null',inplace=True)

I've created a function to allow me to look at the distribution of values in each variable

In [11]:
# function for analysing a variable

def variable_viewer(x):
    values = df[x].value_counts(sort=False)
    proportion = df[x].value_counts(sort=False,normalize='all')
    variable_df = pd.DataFrame({'value_counts': values, 'proportion': proportion})
    return variable_df.sort_values('value_counts', ascending=False)

### Creating new distance features using the long and lat variables

#### Distance from "centre" of London

Making an assertion that trafalgar square is the centre of London.

In [12]:
trafalgar_square = (51.504831314, -0.123499506)

In [13]:
df['distance_from_center'] = df.apply(lambda row: geopy.distance.distance((row['latitude'],row['longitude']),trafalgar_square).km,axis=1)

#### Nearest train station

In [14]:
stations = pd.read_csv('../data/Stations_20180921.csv')

Code to calculate the closest train station to each property and how far away it is in km.

In [16]:
# def station_checker(lat,long):
#     station = ''
#     station_distance = 1000
#     for station_,lat_, long_ in zip(stations.NAME,stations.y,stations.x):
#         calculated_distance = geopy.distance.distance((lat,long),(lat_,long_)).km
#         if calculated_distance < station_distance:
#             station_distance = calculated_distance
#             station = station_
#     return station, station_distance

Storing the results in a dictionary

In [17]:
# station_dict = {'index': [], 'nearest_station': [], 'station_distance': []}


# for i in df.index:
#     station_checker_result = station_checker(df.loc[i]['latitude'],df.loc[i]['longitude'])
#     station_dict['index'].append(i)
#     station_dict['nearest_station'].append(station_checker_result[0])
#     station_dict['station_distance'].append(station_checker_result[1])

# station_df = pd.DataFrame(station_dict)
# station_df.to_csv('../data/station_df.csv')

In [18]:
# creating a dataframe out of the station data

station_df = pd.read_csv('../data/station_df.csv',index_col=1)

station_df.drop('Unnamed: 0',axis=1,inplace=True)

In [19]:
# adding the series to our dataframe

df['nearest_station'] = station_df.nearest_station
df['station_distance'] = station_df.station_distance

### Adding average rental price for the area

data sourced from https://www.ons.gov.uk/peoplepopulationandcommunity/housing/adhocs/12871privaterentalmarketinlondonjanuarytodecember2020

In [22]:
locator = geopy.geocoders.Nominatim(user_agent='myGeocoder',timeout=10)

rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

Code to find out the postcode of the property using the co-ordinates

In [23]:
# def postcode_generator(row):
#     co_ordinates = (row['y'],row['x'])
#     try:
#         location = rgeocode(co_ordinates)
#         postcode = location.raw['address']['postcode'].split()[0]
#         return postcode
#     except:
#         return "error"

# stations['postcode'] = stations.apply(postcode_generator,axis=1)

# stations.to_csv('../data/stations_with_postcode.csv')

In [24]:
stations = pd.read_csv('../data/stations_with_postcode.csv',index_col=0)

In [25]:
# preparing the stations dataframe for the join

stations.rename(columns={'NAME': 'nearest_station'},inplace=True)

stations.set_index('nearest_station',inplace=True)

In [26]:
# joining the stations data with our dataframe

df = df.join(stations, on='nearest_station', how='left')

# removing the columns we don't need

df.drop(['FID','OBJECTID','EASTING','NORTHING','x','y', 'LINES'],axis=1,inplace=True)

df.rename({'NETWORK':'rail_network','Zone':'tfl_zone'},axis=1,inplace=True)

In [27]:
# preparing the rental prices dataframe for the join

rental_prices = pd.read_csv('../data/londonrentalstatisticsq42020.csv')
rental_prices['Mean'] = rental_prices.Mean.apply(lambda x: float(x.replace(',','')))

rental_prices.set_index('Postcode District',inplace=True)
rental_prices.drop('Bedroom Category',axis=1,inplace=True)
rental_prices.rename(columns={'Mean': 'mean_monthly_rent'},inplace=True)

In [28]:
df = df.join(rental_prices,on='postcode',how='left')

Filling the null values in the mean rent column

In [29]:
def rent_filler(row):
    if np.isnan(row['mean_monthly_rent']):
        mean_neighbourhood_rent = df[df.neighbourhood_cleansed==row['neighbourhood_cleansed']]['mean_monthly_rent'].mean()
        return mean_neighbourhood_rent
    else:
        return row['mean_monthly_rent']

In [30]:
df['mean_monthly_rent'] = df.apply(rent_filler,axis=1)

### Removing dollar sign from the price variable

In [31]:
# removing dollar sign from price column and transforming values in to floats

df['price'] = df.price.apply(lambda x: float(x.replace('$','').replace('.00','').replace(',','')))

Removing the properties with zero value for price from the dataframe. From looking at the Airbnb listings, these seem to be properties with zero availability. This is likely why Inside Airbnb were unable to scrape the data for them.

In [32]:
# # Removing the properties with a zero value for price

# price_0 = df[df.price==0]

# price_0.shape[0]

In [33]:
# df.drop(price_0.index,axis=0,inplace=True)

### Engineering the host response time column

In [34]:
df.columns

Index(['id', 'listing_url', 'name', 'description', 'neighborhood_overview',
       'host_id', 'host_since', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'instant_bookable',
       'calculated_host_listings_count', 'distance_from_center',
       'nearest_station', 'station_distance', 'rail_network', 'tfl_zone',
       'postcode', 'mean_monthly_rent'],
      dtype='object')

In [35]:
df.host_response_time.value_counts(normalize='all',dropna=False)

NaN                   0.559928
within an hour        0.224506
within a few hours    0.093560
within a day          0.076951
a few days or more    0.045056
Name: host_response_time, dtype: float64

In [36]:
df.host_response_time.fillna('unknown',inplace=True)

In [37]:
df.host_response_time.value_counts(normalize='all',dropna=False)

unknown               0.559928
within an hour        0.224506
within a few hours    0.093560
within a day          0.076951
a few days or more    0.045056
Name: host_response_time, dtype: float64

In [38]:
df['host_acceptance_rate'] = df.host_acceptance_rate.str.replace('%','').fillna(np.nan).astype('float')

In [39]:
def host_filler(x):
    if not np.isnan(x) and x >= 75:
        return "Above or equal to 75%"
    else:
        return "Below 75%"

In [40]:
df['host_acceptance_rate'] = df.host_acceptance_rate.apply(host_filler)

In [41]:
df['host_response_rate'] = df.host_response_rate.str.replace('%','').fillna(np.nan).astype('float')

In [42]:
df['host_response_rate'] = df.host_response_rate.apply(host_filler)

### Transforming the host_since, first_review and last_review columns

In [43]:
# transforming the host since column to datetime

df['host_since'] = pd.to_datetime(df.host_since)

data_pulled_date = pd.to_datetime('2021-06-04')

It makes more sense to view how long a host has been active on Airbnb for then which date they joined.

In [44]:
# creating a new column to show how many days the host has been active for

def day_transformer(x):
    delta = data_pulled_date - x
    return delta.days

df['host_since'] = df.host_since.apply(day_transformer)

DON'T NEED THE CELL BELOW?

In [ ]:
# # I'll also use the previous function to transform the first_review and last_review columns

# df['first_review'] = pd.to_datetime(df['first_review'])
# df['last_review'] = pd.to_datetime(df['last_review'])

# df['first_review'] = df.first_review.apply(day_transformer)
# df['last_review'] = df.last_review.apply(day_transformer)

### Cleaning and filling the bathrooms_text variable

In [47]:
# converting the bathrooms_text variable from text to a continuous variable

# function to check if string value is numeric

def is_number(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

# function to convert bathrooms_text values

def bathroom_cleaner(x):
    try:
        split = x.lower().split()
        if is_number(split[0]):
            return float(split[0])
        elif 'half-bath' in split:
            return float(0.5)
        else:
            return float(x)
    except:
        return x
    
# replacing old bathrooms_text variable

df['bathrooms_text'] = df.bathrooms_text.apply(bathroom_cleaner)

Filling the bathrooms_text variable with the mean value depending on the room_type and bedrooms value of the property

In [48]:
bathrooms_notnull = df[(df.bathrooms_text.notna()) & (df.bedrooms.notna())][['bedrooms','room_type','bathrooms_text']]

def bathroom_filler(row):
    if np.isnan(row['bathrooms_text']):
        try:
            mean_value = round(bathrooms_notnull[(bathrooms_notnull.room_type==row['room_type']) & (bathrooms_notnull.bedrooms==row['bedrooms'])]['bathrooms_text'].mean())
            return float(mean_value)
        except:
            return row['bathrooms_text'] 
    else:
        return row['bathrooms_text']

df['bathrooms_text'] = df.apply(bathroom_filler,axis=1)

bathrooms_null = df[df.bathrooms_text.isna()]

df.drop(bathrooms_null.index,axis=0,inplace=True)
df['bathrooms_text'] = df.bathrooms_text.apply(lambda x: float(x))

In [49]:
df.rename(columns={'bathrooms_text': 'bathrooms', 'neighbourhood_cleansed': 'neighbourhood'},inplace=True)

### Filling null values in beds and bedrooms columns

In [50]:
# finding out the mean number of bedrooms for each room type

bedrooms_notnull = df[(df.bedrooms.notnull())][['room_type','bedrooms','bathrooms']].copy()

# code to replace null bedroom values with mean values based on room type and number of bathrooms


def bedroom_cleaner(row):
    if np.isnan(row['bedrooms']):
        try:
            mean_value = round(bedrooms_notnull[(bedrooms_notnull.room_type==row['room_type']) & (bedrooms_notnull.bathrooms==row['bathrooms'])]['bedrooms'].mean())
            return mean_value
        except:
            return round(row['bathrooms'])
    else:
        return row['bedrooms']

In [51]:
df['bedrooms'] = df.apply(bedroom_cleaner,axis=1)

In [52]:
# filling the null and zero values in beds

df['beds'] = df.apply(lambda row: row['bedrooms'] if np.isnan(row['beds']) or row['beds']==0 else row['beds'], axis=1)

### Cleaning the property_type column

At the moment the property type column contains too many variables, some with very few values. I'm hoping that a model will perform better if these values are combined in to umbrella categories instead.

In [ ]:
df.property_type.value_counts()

In [ ]:
# new property type categories

property_categories = ['apartment', 'house', 'townhouse', 'condominium',
                       'hotel', 'boutique hotel', 'bed and breakfast', 'loft',
                       'guest suite', 'guesthouse', 'private room', 'aparthotel',
                      'bungalow', 'hostel', 'boat', 'cottage', 'bungalow', 'villa', 'houseboat', 'other']

# function to sort the property column in to new categories

def property_cleaner(x):
    split = x.lower().split()
    if (' ').join(split[-3:]) in property_categories:
        return (' ').join(split[-3:])
    elif (' ').join(split[-2:]) in property_categories:
        return (' ').join(split[-2:])
    elif split[-1]=='houseboat':
        return 'boat'
    elif split[-1] in property_categories:
        return split[-1]
    else:
        return 'other'
    
# apply function to property_type column

df['property_type'] = df.property_type.apply(property_cleaner)

In [ ]:
# creating new columns to show whether the properties have text variables such as description, host_about etc.

df['description_provided'] = df.description.apply(lambda x: 0 if x == 'null' else 1)
df['neighborhood_overview_provided'] = df.neighborhood_overview.apply(lambda x: 0 if x == 'null' else 1)
df['host_about_provided'] = df.host_about.apply(lambda x: 0 if x == 'null' else 1)

In [ ]:
# filling null values in reviews per month with 0

df['reviews_per_month'].fillna(0,inplace=True)

In [ ]:
null_values = pd.DataFrame(df.isnull().sum())
null_values = null_values[null_values[0] != 0]
null_values

### Creating new variables to show length of text data

In [ ]:
nlp_columns = ['name', 'description', 'neighborhood_overview', 'host_about']

def text_counter(text):
    if text != 'null':
        split = text.split()
        return len(split)
    else:
        return 0

for column in nlp_columns:
    df[column+'_length'] = df[column].apply(text_counter)

### Converting true/false columns to binary values

In [ ]:
columns_to_binarise = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
                      'has_availability', 'instant_bookable']

In [ ]:
def binarise(x):
    if x=='t':
        return 1
    elif x=='f':
        return 0
    else:
        return np.nan

In [ ]:
for column in columns_to_binarise:
    df[column] = df[column].apply(binarise)

### Removing properties with no reviews

A lot of the properties with insane outlier values have no reviews. Although this will remove a lot of observations from my data set, it will also provide me with more accurate data to build a model around.

In [ ]:
df[(df.room_type=='Private room')&(df.price>750)].number_of_reviews.describe()

In [ ]:
df[(df.room_type=='Entire home/apt')&(df.price>2000)].number_of_reviews.describe()

Removing all properties that have never been reviewed

In [ ]:
df.drop(df[df.number_of_reviews==0].index,inplace=True)

### Filling in properties that have no host values

In [ ]:
null_sum = df.isnull().sum()

null_sum[(null_sum!=0)]

I'm going to fill the null values in the "host" columns with 0, under the assumption that they are new hosts and do not have the features represented by the other columns.

In [ ]:
# filling the host_since column with mean values

df.host_since.fillna(value=df.host_since.mean(),inplace=True)

In [ ]:
host_columns = ['host_is_superhost','host_listings_count','host_has_profile_pic','host_identity_verified']

for column in host_columns:
    df[column].fillna(0,inplace=True)

# Dummifying the host_verification and amenities columns

In [ ]:
hv_values = []

for hv_list in df.host_verifications:
    if eval(hv_list) != None:
        lst = eval(hv_list)
        for value in lst:
            if value not in hv_values:
                hv_values.append(value)

In [ ]:
hv_values

In [ ]:
# converting the list values in the host_verification column in to binary dummified columns

# compiling all of the unique values within the lists

hv_values = []

for hv_list in df.host_verifications:
    if eval(hv_list) != None:
        lst = eval(hv_list)
        for value in lst:
            if value not in hv_values:
                hv_values.append(value)
            
# creating a dictionary to store the binary values for each value    

hv_dict = {}

for value in hv_values:
    hv_dict[value] = []

# adding the binary values to the dictionary    
    
for hv_list in df.host_verifications:
    if eval(hv_list) != None:
        lst = eval(hv_list)
        for key in hv_dict.keys():
            if key in lst:
                hv_dict[key].append(1)
            else:
                hv_dict[key].append(0)
    else:
        for key in hv_dict.keys():
            hv_dict[key].append(0)
            
# checking that my dictionary has recorded a value for each observation

for key in hv_dict.keys():
    if len(hv_dict[key]) != df.shape[0]:
        print(key, len(hv_dict[key]), "error has occurred")
        
# discarding the values that are present in a very small number of observations         
        
hv_columns = []

for key in hv_dict.keys():
    if sum(hv_dict[key]) >= df.shape[0]*0.01:
        hv_columns.append(key)
        
# adding the columns to the dataframe

for column in hv_columns:
    df["host_verifications_"+column] = hv_dict[column]
    
# dropping the host_verification column from my dataframe

df.drop('host_verifications',axis=1, inplace=True)    

In [ ]:
# converting the list values in the amenities column in to binary dummified columns

amenities_values = []

for amenities_list in df.amenities:
    if eval(amenities_list) != None:
        lst = eval(amenities_list)
        for value in lst:
            if value not in amenities_values:
                amenities_values.append(value)
            
amenities_dict = {}

for value in amenities_values:
    amenities_dict[value] = []    
    
for amenities_list in df.amenities:
    if eval(amenities_list) != None:
        lst = eval(amenities_list)
        for key in amenities_dict.keys():
            if key in lst:
                amenities_dict[key].append(1)
            else:
                amenities_dict[key].append(0)
    else:
        for key in amenities_dict.keys():
            amenities_dict[key].append(0)
            
amenities_columns = []

for key in amenities_dict.keys():
    if sum(amenities_dict[key]) >= df.shape[0]*0.01:
        amenities_columns.append(key)            
        
for column in amenities_columns:
    df["amenities_"+column] = amenities_dict[column]        
    
df.drop('amenities',axis=1, inplace=True)     

### Removing outliers

#### Target Variable

Lots of outliers in the target variable......

I'm going to make the assumption that a lot of these outliers are one of the following:

- erroneously scraped (some of the properties have zero availability, which might have affected whichever software was used to scrape the data)
- the price of the property has been raised by the host to prevent people from renting it (as an alternative to removing the listing?)
- the property has been listed as a joke - see toilet room :)
- the price has been set incorrectly by mistake

In [ ]:
# dropping outliers based on the conditions below

room_outliers = df[(df.room_type=='Private room')&(df.price>1000)]
house_outliers = df[(df.room_type=='Entire home/apt')&(df.price>10000)]

df.drop(room_outliers.index, axis=0, inplace=True)
df.drop(house_outliers.index, axis=0, inplace=True)

In [ ]:
fig,ax = plt.subplots(figsize=(20,10))

sns.boxplot(x=df.price,y=df.room_type,ax=ax)

plt.show()

### Investigating the outliers in the Hotel room and Shared room categories

In [ ]:
df[(df.room_type=='Hotel room')&(df.price>500)].T.head()

These properties seem legitimate!

In [ ]:
df[(df.room_type=='Shared room')&(df.price>400)].T.head()

This property does not seem to be accurately priced. I'm going to remove it from the dataset.

In [ ]:
shared_rooms_drop = [37661065, 17420384, 21425945]

df.drop(df[df.id.isin(shared_rooms_drop)].index,inplace=True)

### Investigating the big price outliers in the Entire home/apt variable

In [ ]:
df[(df.room_type=='Entire home/apt')&(df.price>4000)].sort_values('price',ascending=False)

In [ ]:
entire_houses_drop = [36657089, 11851238, 23706138, 39383869, 7974622, 40518546]


df.drop(df[df.id.isin(entire_houses_drop)].index,inplace=True)

In [ ]:
fig,ax = plt.subplots(figsize=(20,10))

sns.boxplot(x=df.price,y=df.room_type,ax=ax)

plt.show()

### Looking at the outliers in the bathrooms and bedrooms categories

This property gives a price per night for an individual property, yet lists all of the bathrooms and bedrooms for the range of properties the host offers on one one page: https://www.airbnb.com/rooms/43483035 65471
https://www.airbnb.com/rooms/47089782 71819


This property has erroneous listingsL: https://www.airbnb.com/rooms/40222389	58992

In [ ]:
df_drop_outliers = df[(df.listing_url=='https://www.airbnb.com/rooms/43483035')|(df.listing_url=='https://www.airbnb.com/rooms/47089782')|(df.listing_url=='https://www.airbnb.com/rooms/40222389')]

In [ ]:
# removing these properties from the dataset

df.drop(df_drop_outliers.index,axis=0,inplace=True)

### Re-checking for null values

In [ ]:
null_values = pd.DataFrame(df.isnull().sum())
null_values = null_values[null_values[0] != 0]
null_values

### Structured Plan

Perform modelling on features without reviews first? Then model including reviews.

Capture metadata aspects about the reviews? HOw many reviews and over which timeframe?

- Create data dictionary - DONE
- Data Cleaning - DONE
- EDA - partial
- Feature Engineering + Further Data Cleaning - partial
- Linear Regression or Classification? - DONE
- Fit Model on Listings Dataset to Predict Prices - DONE
- Fit Model on Reviews Dataset to Predict Prices - DONE
- Combine Both to Predict Prices - DONE
- Visualise findings - use the Tableau location function
- Perform Clustering on the Reviews - what insights can we gather? Create word clouds
- Predict reviews based on NLP of reviews
- What are people looking for when they stay at an Airbnb?
- Which neighborhoods are the most popular? Which are the most expensive?
- Can we see any trends on where people like to stay?
- Are there other features that we can use from different datasets

When transforming data - do train and test split before transforming. This means that your model isn't already aware words that appear in your test set. You need to turn-off drop first, though, and set the parameter to ignore any unknown words.

Notes:

- can we apply the model to other cities?

To-Do List

Data Cleaning:

- use median values rather than mean values (mean values will be swayed more by outliers)
- simplify the categorisation of the property type variable
- apply lower and higher limits to the price variable to deal with outliers
- simplify the amenities + host binarised variables
- create a new column to show the average property price for each host_id
- bring in geographical proximity of attractions as target variables

Variable Transformation:

- look at distributions of continuous/discrete variables - do they need transforming?
- look in to log transforming the continuous variables (naive-Bayes lessons)

Modelling:

- review the use of NLP - could we instead look at key words within the variables? This might be a better option for the title of the 
- can we use neural networks?

good visualisations: https://towardsdatascience.com/predicting-airbnb-prices-with-deep-learning-part-2-how-to-improve-your-nightly-price-50ea8bc2bd29